# Melatih RL untuk Menyeimbangkan Cartpole

Notebook ini adalah bagian dari [Kurikulum AI untuk Pemula](http://aka.ms/ai-beginners). Notebook ini terinspirasi dari [tutorial resmi PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) dan [implementasi Cartpole PyTorch ini](https://github.com/yc930401/Actor-Critic-pytorch).

Dalam contoh ini, kita akan menggunakan RL untuk melatih sebuah model agar dapat menyeimbangkan sebuah tiang pada kereta yang dapat bergerak ke kiri dan ke kanan pada skala horizontal. Kita akan menggunakan lingkungan [OpenAI Gym](https://www.gymlibrary.ml/) untuk mensimulasikan tiang tersebut.

> **Catatan**: Anda dapat menjalankan kode pelajaran ini secara lokal (misalnya dari Visual Studio Code), di mana simulasi akan terbuka di jendela baru. Saat menjalankan kode secara online, Anda mungkin perlu melakukan beberapa penyesuaian pada kode, seperti yang dijelaskan [di sini](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Kita akan memulai dengan memastikan Gym telah terinstal:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Sekarang mari kita buat lingkungan CartPole dan lihat bagaimana cara mengoperasikannya. Sebuah lingkungan memiliki properti berikut:

* **Action space** adalah kumpulan aksi yang dapat kita lakukan pada setiap langkah simulasi
* **Observation space** adalah ruang pengamatan yang dapat kita amati


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Mari kita lihat bagaimana simulasi bekerja. Loop berikut menjalankan simulasi hingga `env.step` tidak mengembalikan tanda penghentian `done`. Kita akan memilih aksi secara acak menggunakan `env.action_space.sample()`, yang berarti eksperimen kemungkinan besar akan gagal dengan cepat (lingkungan CartPole berakhir ketika kecepatan CartPole, posisinya, atau sudutnya berada di luar batas tertentu).

> Simulasi akan terbuka di jendela baru. Anda dapat menjalankan kode beberapa kali dan melihat bagaimana hasilnya.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Anda dapat melihat bahwa observasi mengandung 4 angka. Angka-angka tersebut adalah:
- Posisi kereta
- Kecepatan kereta
- Sudut tiang
- Laju rotasi tiang

`rew` adalah hadiah yang kita terima di setiap langkah. Dalam lingkungan CartPole, Anda mendapatkan 1 poin untuk setiap langkah simulasi, dan tujuannya adalah memaksimalkan total hadiah, yaitu waktu CartPole dapat menjaga keseimbangan tanpa jatuh.

Selama pembelajaran penguatan, tujuan kita adalah melatih **kebijakan** $\pi$, yang untuk setiap keadaan $s$ akan memberi tahu kita tindakan $a$ yang harus diambil, sehingga pada dasarnya $a = \pi(s)$.

Jika Anda menginginkan solusi probabilistik, Anda dapat menganggap kebijakan sebagai pengembalian sekumpulan probabilitas untuk setiap tindakan, yaitu $\pi(a|s)$ akan berarti probabilitas bahwa kita harus mengambil tindakan $a$ pada keadaan $s$.

## Metode Policy Gradient

Dalam algoritma RL yang paling sederhana, yang disebut **Policy Gradient**, kita akan melatih jaringan saraf untuk memprediksi tindakan berikutnya.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Kami akan melatih jaringan dengan menjalankan banyak eksperimen, dan memperbarui jaringan kami setelah setiap percobaan. Mari kita definisikan sebuah fungsi yang akan menjalankan eksperimen dan mengembalikan hasilnya (yang disebut **jejak**) - semua keadaan, tindakan (dan probabilitas yang direkomendasikan), serta hadiah:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Anda dapat menjalankan satu episode dengan jaringan yang belum dilatih dan mengamati bahwa total hadiah (alias durasi episode) sangat rendah:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Salah satu aspek rumit dari algoritma policy gradient adalah menggunakan **reward yang didiskon**. Idenya adalah kita menghitung vektor total reward pada setiap langkah permainan, dan selama proses ini kita mendiskon reward awal menggunakan beberapa koefisien $gamma$. Kita juga menormalkan vektor yang dihasilkan, karena kita akan menggunakannya sebagai bobot untuk memengaruhi pelatihan kita:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Sekarang mari kita mulai pelatihan! Kita akan menjalankan 300 episode, dan pada setiap episode kita akan melakukan hal berikut:

1. Jalankan eksperimen dan kumpulkan jejaknya.
2. Hitung perbedaan (`gradients`) antara tindakan yang diambil dan probabilitas yang diprediksi. Semakin kecil perbedaannya, semakin yakin kita bahwa tindakan yang diambil adalah tindakan yang benar.
3. Hitung hadiah yang didiskon dan kalikan `gradients` dengan hadiah yang didiskon - ini akan memastikan bahwa langkah-langkah dengan hadiah lebih tinggi memiliki pengaruh lebih besar pada hasil akhir dibandingkan dengan langkah-langkah dengan hadiah lebih rendah.
4. Tindakan target yang diharapkan untuk jaringan saraf kita sebagian akan diambil dari probabilitas yang diprediksi selama proses berjalan, dan sebagian dari `gradients` yang dihitung. Kita akan menggunakan parameter `alpha` untuk menentukan sejauh mana `gradients` dan hadiah diperhitungkan - ini disebut *learning rate* dari algoritma penguatan.
5. Akhirnya, kita melatih jaringan kita pada keadaan dan tindakan yang diharapkan, lalu mengulangi prosesnya.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Sekarang mari jalankan episode dengan rendering untuk melihat hasilnya:


In [ ]:
_ = run_episode(render=True)

Semoga Anda bisa melihat bahwa tiang sekarang dapat seimbang dengan cukup baik!

## Model Actor-Critic

Model Actor-Critic adalah pengembangan lebih lanjut dari policy gradients, di mana kita membangun jaringan neural untuk mempelajari kebijakan (policy) dan estimasi reward secara bersamaan. Jaringan ini akan memiliki dua output (atau Anda bisa melihatnya sebagai dua jaringan terpisah):
* **Actor** akan merekomendasikan tindakan yang harus diambil dengan memberikan distribusi probabilitas keadaan, seperti pada model policy gradient.
* **Critic** akan memperkirakan apa reward yang akan diperoleh dari tindakan tersebut. Critic mengembalikan total estimasi reward di masa depan pada keadaan tertentu.

Mari kita definisikan model seperti itu:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Kita perlu sedikit memodifikasi fungsi `discounted_rewards` dan `run_episode` kita:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Sekarang kita akan menjalankan loop pelatihan utama. Kita akan menggunakan proses pelatihan jaringan manual dengan menghitung fungsi kerugian yang sesuai dan memperbarui parameter jaringan:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Poin Penting

Kita telah melihat dua algoritma RL dalam demo ini: simple policy gradient, dan actor-critic yang lebih canggih. Anda dapat melihat bahwa algoritma-algoritma tersebut bekerja dengan konsep abstrak seperti state, action, dan reward - sehingga algoritma ini dapat diterapkan pada berbagai jenis lingkungan yang sangat berbeda.

Reinforcement learning memungkinkan kita mempelajari strategi terbaik untuk menyelesaikan masalah hanya dengan melihat reward akhir. Fakta bahwa kita tidak memerlukan dataset berlabel memungkinkan kita untuk mengulangi simulasi berkali-kali guna mengoptimalkan model kita. Namun, masih ada banyak tantangan dalam RL, yang mungkin akan Anda pelajari jika Anda memutuskan untuk lebih mendalami bidang AI yang menarik ini.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
